# Daqss API Tutorial

## Overview

An application program interface (API) is a set of routines, protocols, and tools for building software applications. A good APi makes it easier to develop a program by providing the building blocks. A programmer then puts those blocks together.

The Daqss API provides an easy way to read and write data to/from the remote server. It is protected by key/password combinations and allows Users read-only access. The following describes how to use the Python wrapper that is built on top of the Daqss API.

## Contents

  1. [Installation](#installation)
  2. [API Credentials](#api-credentials)
  3. [API Endpoints](#api-endpoints)
  4. [Using the Python API Wrapper](#how-to-use)

<a id="#installation"></a>

## Installation

To install the python-daqss python API wrapper, download from the Github repository here, or clone the repository to your computer.

### Download and unzip

    >>> wget https://github.com/dhhagan/python-daqss/archive/master.zip
    >>> unzip master.zip
    >>> cd python-daqss/
    >>> sudo python setup.py install

### Install via git

    >>> git clone https://github.com/dhhagan/python-daqss.git
    >>> cd python-daqss/
    >>> sudo python setup.py install
    
You should now have the python API wrapper installed on your computer.

<a id="#api-credentials"></a>

## API Credentials

The Daqss API is protected using a key, password system that prevents unwanted people from injecting data into the database, or deleting entries. In order to use the API to communicate with the database, you must first request your own set of API credentials.

As of right now, you must email david@davidhhagan.com for API credentials!

## API Endpoints

API endpoints at their most basic level, share a common structure; essentially, you send a url that is formatted in a specific way, accompanied by your credentials, and recieve data (or an error!) in return in [json](http://json.org/) format. All of the available API endpoints are layed out in detail in the [full documentation](#). 

For example, say we wanted to request the data for one of the alphasense gas sensors. The documentation looks something like this:

### Resource URL

   * **GET api/< api-version >/alphasense/< sn >/data/date/< basedate >/< period >**
   * **GET api/< api-version >/alphasense/< sn >/data/date/< basedate >/< enddate >**


| api-version | The API version. Currently 1.0 |
|:-|:-|
| sn | The serial number for the alphasense sensor of interest|
| basedate | The beginning date for the data you want to recieve in the format **YYYY-MM-DD** |
| period | The period (length) for which you want data beginning at the basedate. One of **1d, 2w, 4m, 1y** |
| enddate | The ending date for the data you want to recieve in the format **YYYY-MM-DD** |


<br />
Thus, once you build the resource url, you might end up with something like this: ** api/v1.0/alphasense/A123/data/date/2015-01-01/1w**

## Using the python API wrapper

Luckily, a python wrapper has been written around the Daqss API to make things easy for everyone! 

In [1]:
import daqss
from daqss import Daqss
import pandas as pd

key = 'DHDNAG3E43AV'
pswd = 'S9L5NNDLKAJU'

print ("daqss library version: {0}".format(daqss.__version__))

daqss library version: 0.0.11


### Set up the class instance of Daqss

In [2]:
# Set up an instance of the class Daqss with keyword arguments key and password
api = Daqss(key, pswd)

# Ping to see if you have a connection
api.ping()

True

### Your first API call

To begin, let's try grabbing all of the nodes from the database. Instead of worrying about the API endpoint, we can just call `api.get_nodes()` and you will get what you desire!

Everything is in json format, so you will need to learn how to deal with that..don't worry, it's easy!

In [9]:
status, req = api.get_nodes()

print ("The Status Code: {0}".format(status))
print ("The response: \n")
req['nodes']

The Status Code: 200
The response: 



[{u'co_sensor': u'http://45.55.154.140/api/v1.0/alphasense/1',
  u'description': u'DAQSS001',
  u'last_updated': u'Mon, 15 Jun 2015 10:00:02 GMT',
  u'latitude': 0.0,
  u'longitude': 0.0,
  u'no2_sensor': u'http://45.55.154.140/api/v1.0/alphasense/3',
  u'no_sensor': u'http://45.55.154.140/api/v1.0/alphasense/2',
  u'node_sn': u'DAQSS001',
  u'o3_sensor': u'http://45.55.154.140/api/v1.0/alphasense/4',
  u'opc': None,
  u'rht_sensor': u'http://45.55.154.140/api/v1.0/rht/1',
  u'url': u'http://45.55.154.140/api/v1.0/nodes/DAQSS001'},
 {u'co_sensor': u'http://45.55.154.140/api/v1.0/alphasense/5',
  u'description': u'DAQSS002',
  u'last_updated': u'Mon, 01 Jun 2015 10:00:02 GMT',
  u'latitude': 0.0,
  u'longitude': 0.0,
  u'no2_sensor': u'http://45.55.154.140/api/v1.0/alphasense/7',
  u'no_sensor': u'http://45.55.154.140/api/v1.0/alphasense/6',
  u'node_sn': u'DAQSS002',
  u'o3_sensor': u'http://45.55.154.140/api/v1.0/alphasense/8',
  u'opc': None,
  u'rht_sensor': u'http://45.55.154.140/a

### Making the data more usable

Let's take advantage of the amazing library [pandas]() and DataFrame's so we can easily manipulate the data and export to csv if we want to.

We will do so by reading the `nodes` portion of the json response into a pandas DataFrame. All of the column names and everything should be automatically set.

In [14]:
# Read into a DataFrame
df = pd.DataFrame.from_dict(req['nodes'])

# Print the info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 12 columns):
co_sensor       5 non-null object
description     5 non-null object
last_updated    5 non-null object
latitude        5 non-null float64
longitude       5 non-null float64
no2_sensor      5 non-null object
no_sensor       5 non-null object
node_sn         5 non-null object
o3_sensor       5 non-null object
opc             0 non-null object
rht_sensor      5 non-null object
url             5 non-null object
dtypes: float64(2), object(10)
memory usage: 520.0+ bytes


### Exporting

Now we are going to export this data to a csv for whatever reason!

In [15]:
df.to_csv("Nodes.csv")

## Getting the Data!

Now, let's download some data and export it. We are going to download one months worth of data for the alphasense sensor associated with Node 3. From the config file, we know that the SN for this O3 sensor is **203490202**. According to the documentation for the API wrapper, we can get the alphasense data using the following api call:

**get_alphasense_data(self, sn, basedate = None, enddate = None, period = None)**

Let's grab the O3 data for the time period of 1 month beginning on May 1, 2015.

In [26]:
status, data = api.get_alphasense_data('203490202', basedate = '2015-05-01', period='6m')

# read into a dataframe
df = pd.DataFrame.from_dict(data['data'])

# Re-index so we have a nice datetime index to work with
df.index = pd.to_datetime(df['timestamp'])

# Export to csv!
df.to_csv("O3 Node 3 May 2015.csv")

# Print the info
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 118873 entries, 2015-05-01 00:00:03 to 2015-06-18 10:52:29
Data columns (total 5 columns):
ae_volts         118873 non-null float64
concentration    0 non-null object
sensor           118873 non-null object
timestamp        118873 non-null object
we_volts         118873 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.4+ MB


In [ ]:
status, data = api.get_rht_data('RHT001', basedate = '2015-05-01', period = '1w')

# read into a dataframe
df = pd.DataFrame.from_dict(data['data'])

# Re-index so we have a nice datetime index to work with
df.index = pd.to_datetime(df['timestamp'])

# Print the info
df.info()